# Routing Topology: OSPF using FRRouting

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


## Step 1:  Configure the Environment


## Step 2: Import the FABLlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

-----------------------------------  --------------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /Users/pruth/work/fabric_config/fabric_token.json
project_id                           990d8a8b-7e50-4d13-a3be-0f133ffa8653
bastion_username                     pruth_0031379841
bastion_key_filename                 /Users/pruth/work/fabric_config/fabric_bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /Users/pruth/work/fabric_config/slice-public-key
slice_private_key_file               /Users/pruth/work/fabric_config/slice-private-key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  -------------------

## Step 3 (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MICH         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  320/320  2560/2560  116400/116400  635/635                 2/2                             4/4                            16/16               4/4               5/5
TACC        10  316/320  2520/2560  116370/116400  631/635                 2/2                             4/4                            16/16               4/4               6/6
WASH         6  186/192  1512/1536  60570/60600    376/381                 2/2

## Step 4: Create the Experiment Slice

The following creates private layer 2 networks on three sites including a OSPF gateway routers that propogate routes acrross the topology. 


In [3]:
slice_name = 'OSPF_Routing_Topology'

[site1,site2,site3] = fablib.get_random_sites(count=3)
print(f"Sites: {site1},{site2},{site3}")

router_base_name='router'
router_link_base_name='router_link'

node_base_name='node'
local_network_base_name='net_local'

site_node_count=2


router1_name = f"{router_base_name}1"
router2_name = f"{router_base_name}2"
router3_name = f"{router_base_name}3"

#Create Router Links
route_link1_name = f'{router_link_base_name}1'
route_link2_name = f'{router_link_base_name}2'
route_link3_name = f'{router_link_base_name}3'

#Create Site Local Networks
site1_local_net_name = f'{local_network_base_name}1'
site2_local_net_name = f'{local_network_base_name}2'
site3_local_net_name = f'{local_network_base_name}3'
    


Sites: UTAH,SALT,DALL


In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    slice.load("ospf.graphml")

    #Submit Slice Request
    slice_id = slice.submit()

except Exception as e:
    print(f"Slice Fail: {e}")


---------------  ------------------------------------
Slice Name       OSPF_Routing_Topology
Slice ID         455be5cb-4ed6-4b02-b0bf-19f9fb0c0c10
Slice State      StableOK
Lease End (UTC)  2022-07-18 15:56:04 +0000
---------------  ------------------------------------

Retry: 16, Time: 193 sec

ID                                    Name          Site    Host                          Cores    RAM    Disk  Image            Management IP                           State    Error
------------------------------------  ------------  ------  --------------------------  -------  -----  ------  ---------------  --------------------------------------  -------  -------
f6935426-62aa-43d0-9172-5d05f6498337  router1       WASH    wash-w2.fabric-testbed.net        2      8      10  default_rocky_8  2001:400:a100:3020:f816:3eff:fee5:f65c  Active
85c3411a-9439-4b0d-9530-79572fd7c629  router2       SALT    salt-w1.fabric-testbed.net        2      8      10  default_rocky_8  2001:400:a100:3010:f816:3ef

## Step 5: Observe the Slice's Attributes

### Print the slice

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

---------------  ------------------------------------
Slice Name       OSPF_Routing_Topology
Slice ID         455be5cb-4ed6-4b02-b0bf-19f9fb0c0c10
Slice State      StableOK
Lease End (UTC)  2022-07-18 15:56:04 +0000
---------------  ------------------------------------


### Print the Node List

In [6]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID                                    Name          Site    Host                          Cores    RAM    Disk  Image            Management IP                           State    Error
------------------------------------  ------------  ------  --------------------------  -------  -----  ------  ---------------  --------------------------------------  -------  -------
f6935426-62aa-43d0-9172-5d05f6498337  router1       WASH    wash-w2.fabric-testbed.net        2      8      10  default_rocky_8  2001:400:a100:3020:f816:3eff:fee5:f65c  Active
85c3411a-9439-4b0d-9530-79572fd7c629  router2       SALT    salt-w1.fabric-testbed.net        2      8      10  default_rocky_8  2001:400:a100:3010:f816:3eff:fe06:f529  Active
0d6d12a2-f77c-40fa-8dac-b5bb422a9f76  router3       DALL    dall-w3.fabric-testbed.net        2      8      10  default_rocky_8  2001:400:a100:3000:f816:3eff:fe8d:fa12  Active
27b420aa-16b6-4280-8f01-f6d808962059  node_site1_1  WASH    wash-w2.fabric-testbed.net        2      8

### Print the Node Details

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------------------------------------
ID                 f6935426-62aa-43d0-9172-5d05f6498337
Name               router1
Cores              2
RAM                8
Disk               10
Image              default_rocky_8
Image Type         qcow2
Host               wash-w2.fabric-testbed.net
Site               WASH
Management IP      2001:400:a100:3020:f816:3eff:fee5:f65c
Reservation State  Active
Error Message
SSH Command        ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@2001:400:a100:3020:f816:3eff:fee5:f65c
-----------------  ------------------------------------------------------------------------------------------------------------------------------------------------------
-----------------  ------------------------------------------------------------------------------------------------

### Print the Node SSH Commands

In [8]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

router1: ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@2001:400:a100:3020:f816:3eff:fee5:f65c
router2: ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@2001:400:a100:3010:f816:3eff:fe06:f529
router3: ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@2001:400:a100:3000:f816:3eff:fe8d:fa12
node_site1_1: ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@2001:400:a100:3020:f816:3eff:fea5:fdcb
node_site1_2: ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@2001:400:a100:3020:f816:3eff:fe70:f01a
node_site2_1: ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@2001:400:a100:3010:f816:3eff:fe37:7378
node_site2_2: ssh -i /Use

### Print the Interfaces

In [9]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name                  Node          Network         Bandwidth  VLAN    MAC                Physical OS Interface    OS Interface
--------------------  ------------  ------------  -----------  ------  -----------------  -----------------------  --------------
router1-nic_local-p1  router1       net_local1              0          06:2C:82:9C:93:8F  eth2                     eth2
router1-nic1-p1       router1       router_link1            0          06:40:40:74:22:A4  eth3                     eth3
router1-nic2-p1       router1       router_link3            0          06:0A:F5:34:EB:6D  eth1                     eth1
router2-nic2-p1       router2       router_link1            0          02:25:EF:C4:AE:D0  eth2                     eth2
router2-nic_local-p1  router2       net_local2              0          02:09:16:54:61:EB  eth1                     eth1
router2-nic1-p1       router2       router_link2            0          02:43:FC:D8:5D:CF  eth3                     eth3
router3-nic1-p1       

## Step 6: Configure IP Addresses

This experiment includes six different networks, each of which need to have a distinct subnet. The following cells show how to pick IPv4 subnets for each network and assign addresses from those subnets to the appropriate interfaces on the nodes and routers.


### Pick Subnets

Pick the subnet for the routing links. Each routing link connects a pair of routers. Although these links always have exactly two interfaces, we choose a /24 subnet for easy readability. For each link we create a subnet and a list of available IPs for that subnet. These will be used later to configure the router interfaces connected to these links. 

In [10]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    # Route Link Subnets
    route_link1_subnet = IPv4Network("192.168.101.0/24")
    route_link1_available_ips = list(route_link1_subnet)[1:]
    route_link1_addr1 = route_link1_available_ips.pop(0)
    route_link1_addr2 = route_link1_available_ips.pop(0)
    
    route_link2_subnet = IPv4Network("192.168.102.0/24")
    route_link2_available_ips = list(route_link2_subnet)[1:]
    route_link2_addr1 = route_link2_available_ips.pop(0)
    route_link2_addr2 = route_link2_available_ips.pop(0)
    
    route_link3_subnet = IPv4Network("192.168.103.0/24")
    route_link3_available_ips = list(route_link3_subnet)[1:]
    route_link3_addr1 = route_link3_available_ips.pop(0)
    route_link3_addr2 = route_link3_available_ips.pop(0)
    
    print(f"Router Link 1: subnet: {route_link1_subnet}, addr1: {route_link1_addr1}, addr2: {route_link1_addr2}")
    print(f"Router Link 2: subnet: {route_link2_subnet}, addr1: {route_link2_addr1}, addr2: {route_link2_addr2}")
    print(f"Router Link 3: subnet: {route_link3_subnet}, addr1: {route_link3_addr1}, addr2: {route_link3_addr2}")

except Exception as e:
    print(f"Exception: {e}")
   

Router Link 1: subnet: 192.168.101.0/24, addr1: 192.168.101.1, addr2: 192.168.101.2
Router Link 2: subnet: 192.168.102.0/24, addr1: 192.168.102.1, addr2: 192.168.102.2
Router Link 3: subnet: 192.168.103.0/24, addr1: 192.168.103.1, addr2: 192.168.103.2


Pick the subnet for the local networks.  The /24 subnets can support up to 254 locally connected nodes plus the gateway. 


In [11]:
try:
    # Local Subnets
    net_local1_subnet = IPv4Network("192.168.1.0/24")
    net_local1_available_ips = list(net_local1_subnet)[1:]
    net_local1_gateway = net_local1_available_ips.pop(0)
        
    net_local2_subnet = IPv4Network("192.168.2.0/24")
    net_local2_available_ips = list(net_local2_subnet)[1:]
    net_local2_gateway = net_local2_available_ips.pop(0)


    net_local3_subnet = IPv4Network("192.168.3.0/24")
    net_local3_available_ips = list(net_local3_subnet)[1:]
    net_local3_gateway = net_local3_available_ips.pop(0)

    print(f"Site1: subnet: {net_local1_subnet}, gateway: {net_local1_gateway}")
    print(f"Site2: subnet: {net_local2_subnet}, gateway: {net_local2_gateway}")
    print(f"Site3: subnet: {net_local3_subnet}, gateway: {net_local3_gateway}")

except Exception as e:
    print(f"Exception: {e}")

Site1: subnet: 192.168.1.0/24, gateway: 192.168.1.1
Site2: subnet: 192.168.2.0/24, gateway: 192.168.2.1
Site3: subnet: 192.168.3.0/24, gateway: 192.168.3.1


### Configure Router IPs

Add the IPs to the corrisponing interfaces.

In [12]:
import traceback
try:
    # Config Router1 IPs
    router1 = slice.get_node(name=router1_name)  
    router1_iface1 = router1.get_interface(network_name=route_link1_name)  
    router1_iface2 = router1.get_interface(network_name=route_link3_name)  
    router1_local_iface = router1.get_interface(network_name=site1_local_net_name)     
    router1_iface1.ip_addr_add(addr=route_link1_addr1, subnet=route_link1_subnet)
    router1_iface2.ip_addr_add(addr=route_link3_addr2, subnet=route_link3_subnet)
    router1_local_iface.ip_addr_add(addr=net_local1_gateway, subnet=net_local1_subnet)
    
    # Config Router2 IPs
    router2 = slice.get_node(name=router2_name)  
    router2_iface1 = router2.get_interface(network_name=route_link2_name)  
    router2_iface2 = router2.get_interface(network_name=route_link1_name)  
    router2_local_iface = router2.get_interface(network_name=site2_local_net_name)     
    router2_iface1.ip_addr_add(addr=route_link2_addr1, subnet=route_link2_subnet)
    router2_iface2.ip_addr_add(addr=route_link1_addr2, subnet=route_link1_subnet)
    router2_local_iface.ip_addr_add(addr=net_local2_gateway, subnet=net_local2_subnet)
    
    # Config Router3 IPs
    router3 = slice.get_node(name=router3_name) 
    router3_iface1 = router3.get_interface(network_name=route_link3_name)  
    router3_iface2 = router3.get_interface(network_name=route_link2_name)  
    router3_local_iface = router3.get_interface(network_name=site3_local_net_name)     
    router3_iface1.ip_addr_add(addr=route_link3_addr1, subnet=route_link3_subnet)
    router3_iface2.ip_addr_add(addr=route_link2_addr2, subnet=route_link2_subnet)
    router3_local_iface.ip_addr_add(addr=net_local3_gateway, subnet=net_local3_subnet)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()
    

(Optional) Print the network interface configuration for each router.

In [13]:
try:
    for router in [router1, router2, router3]:
        print(f'{router.get_name()}:')
        stdout, stderr = router.execute(f'ip addr list')
        print (stdout)
        print (stderr)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

router1:
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:e5:f6:5c brd ff:ff:ff:ff:ff:ff
    inet 10.20.5.19/23 brd 10.20.5.255 scope global dynamic noprefixroute eth0
       valid_lft 86109sec preferred_lft 86109sec
    inet6 2001:400:a100:3020:f816:3eff:fee5:f65c/64 scope global dynamic noprefixroute 
       valid_lft 86196sec preferred_lft 14196sec
    inet6 fe80::f816:3eff:fee5:f65c/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:0a:f5:34:eb:6d brd ff:ff:ff:ff:ff:ff
 

### Configure Local Node IPs

Configure the local nodes with addresses from the local subnet available address list. Add routes to the other local subnets via the local gateway.  

Add the router link subnets if you want to access the routers (i.e. if you want to `ping` the routers or find paths with `tracepath`)

Collect a list of local dataplane IPs to target for testing

In [14]:
local_dataplane_ips = {}

In [15]:
try:    
    #Create Site1 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_site1_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local1_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site1_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local1_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local2_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=net_local3_subnet, gateway=net_local1_gateway)
        
        #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local1_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr
        
        print(f"Node {name} dataplane IP: {node_addr}")    
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    

Node node_site1_1 dataplane IP: 192.168.1.2
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:55:70:3b:a3:4f brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/24 scope global eth1
       valid_lft forever preferred_lft forever

Node node_site1_2 dataplane IP: 192.168.1.3
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:c9:b7:7a:b3:9e brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.3/24 scope global eth1
       valid_lft forever preferred_lft forever



In [16]:
try:
    #Create Site2 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_site2_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local2_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site2_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local2_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local1_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=net_local3_subnet, gateway=net_local2_gateway)
        
         #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local2_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr

        print(f"Node {name} dataplane IP: {node_addr}")
        stdout, stderr = node.execute(f'ip addr list')
        print (stdout)
        
        stdout, stderr = ndoe.execute(f'ip addr show {node_iface.get_os_interface()}')
        print (stdout)
except Exception as e:
    print(f"Exception: {e}")

Node node_site2_1 dataplane IP: 192.168.2.2
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:37:73:78 brd ff:ff:ff:ff:ff:ff
    inet 10.20.4.36/23 brd 10.20.5.255 scope global dynamic noprefixroute eth0
       valid_lft 86083sec preferred_lft 86083sec
    inet6 2001:400:a100:3010:f816:3eff:fe37:7378/64 scope global dynamic noprefixroute 
       valid_lft 86197sec preferred_lft 14197sec
    inet6 fe80::f816:3eff:fe37:7378/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:9e:

In [17]:
try:
    #Create Site3 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_site3_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local3_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site3_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local3_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local1_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=net_local2_subnet, gateway=net_local3_gateway)
        
        #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local3_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr
        
        print(f"Node {name} dataplane IP: {node_addr}")
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

Node node_site3_1 dataplane IP: 192.168.3.2
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:e9:54:ee:5a:01 brd ff:ff:ff:ff:ff:ff
    inet 192.168.3.2/24 scope global eth1
       valid_lft forever preferred_lft forever

Node node_site3_2 dataplane IP: 192.168.3.3
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0e:c8:50:3c:20:e5 brd ff:ff:ff:ff:ff:ff
    inet 192.168.3.3/24 scope global eth1
       valid_lft forever preferred_lft forever



##  Step 7: Configure FRRouting on each router. 

This complex configuration is handled through a bash script `frr_config.sh` that resides in the folder containing this notebook. The script is executed by, first, uploading the script with the `node.upload_file()` FABLib method.  Then the script is executed using the `node.execute()` FABLib method.  Note that the script passes the OS interfaces names and configured IPs as arguments from the notebook to the script.  

These scripts take a while to run. You may wish to use a separate terminal window to ssh to the routers and tail the log file to watch the progess with: `tail -F frr_config.log`.

In [18]:
try: 
    #thread1 = execute.thread()
    #stdout, stderr = thread1.result()
    
    # Config Router1
    print('Config Router1')
    router1.upload_file('./frr_config.sh','frr_config.sh')
    router1_config_thread = router1.execute_thread(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router1_iface1.get_os_interface()} {route_link1_addr1} {router1_iface2.get_os_interface()} {route_link3_addr2} {router1_local_iface.get_os_interface()} {net_local1_gateway} 192.168.0.0')

    # Config Router2
    print('Config Router2')
    router2.upload_file('./frr_config.sh','frr_config.sh')   
    router2_config_thread = router2.execute_thread(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router2_iface1.get_os_interface()} {route_link2_addr1} {router2_iface2.get_os_interface()} {route_link1_addr2} {router2_local_iface.get_os_interface()} {net_local2_gateway} 192.168.0.0')
    
    # Config Router3
    print('Config Router3')
    router3.upload_file('./frr_config.sh','frr_config.sh')
    router3_config_thread = router3.execute_thread(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router3_iface1.get_os_interface()} {route_link3_addr1} {router3_iface2.get_os_interface()} {route_link2_addr2} {router3_local_iface.get_os_interface()} {net_local3_gateway} 192.168.0.0')

    #Join Threads
    print(f"Joining Threads")
    stdout, stderr = router1_config_thread.result() 
    print(f"Router1: ", stdout, stderr)
    stdout, stderr = router2_config_thread.result() 
    print(f"Router2: ", stdout, stderr)
    stdout, stderr = router3_config_thread.result() 
    print(f"Router3: ", stdout, stderr)

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()            
     

Config Router1
Config Router2
Config Router3
Joining Threads
Router1:  Importing GPG key 0x6D745A60:
 Userid     : "Release Engineering <infrastructure@rockylinux.org>"
 Fingerprint: 7051 C470 A929 F454 CEBE 37B7 15AF 5DAC 6D74 5A60
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-rockyofficial
 
Router2:  Importing GPG key 0x6D745A60:
 Userid     : "Release Engineering <infrastructure@rockylinux.org>"
 Fingerprint: 7051 C470 A929 F454 CEBE 37B7 15AF 5DAC 6D74 5A60
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-rockyofficial
 
Router3:  Importing GPG key 0x6D745A60:
 Userid     : "Release Engineering <infrastructure@rockylinux.org>"
 Fingerprint: 7051 C470 A929 F454 CEBE 37B7 15AF 5DAC 6D74 5A60
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-rockyofficial
 


## Step 8: Run the Experiment

We will just test `ping` RTT and look at `tracepath`. Your experiment should be more interesting!

Notice that if you run this quickly and repeatedly run this test against a specific target, you may see changes to the tracepath.  Initially the ping may even fail.  Why do you think this is happening?


In [22]:
try:
    source_node_name =  f'{node_base_name}_site1_1'
   
    source_node = slice.get_node(name=source_node_name)
    for node_name,target_ip in local_dataplane_ips.items():
        print(f"Testing target node: {node_name}, target IP: {target_ip}")
    
        stdout, stderr = node.execute(f'ping -c 5 {target_ip}')
        print (stdout, stderr)

        stdout, stderr = node.execute(f'tracepath {target_ip}')
        print (stdout, stderr)
    
except Exception as e:
    print(f"Exception: {e}")

Testing target node: node_site1_1, target IP: 192.168.1.2
PING 192.168.1.2 (192.168.1.2) 56(84) bytes of data.
64 bytes from 192.168.1.2: icmp_seq=1 ttl=62 time=29.10 ms
64 bytes from 192.168.1.2: icmp_seq=2 ttl=62 time=29.6 ms
64 bytes from 192.168.1.2: icmp_seq=3 ttl=62 time=29.6 ms
64 bytes from 192.168.1.2: icmp_seq=4 ttl=62 time=29.6 ms
64 bytes from 192.168.1.2: icmp_seq=5 ttl=62 time=29.6 ms

--- 192.168.1.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4006ms
rtt min/avg/max/mdev = 29.608/29.701/29.994/0.239 ms
 
 1?: [LOCALHOST]                      pmtu 1500
 1:  192.168.3.1                                           0.255ms 
 1:  192.168.3.1                                           0.084ms 
 2:  192.168.103.2                                        29.636ms 
 3:  192.168.1.2                                          29.630ms reached
     Resume: pmtu 1500 hops 3 back 3 
 
Testing target node: node_site1_2, target IP: 192.168.1.3
PING 192.168.1.3 (

In [ ]:
try:
    slice.save("ospf.graphml")
except Exception as e:
    print(f"Exception: {e}")

## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

In [24]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")

Exception: 'NoneType' object has no attribute 'delete'
